<a href="https://colab.research.google.com/github/huiesoo/esaa/blob/main/24-1/0412.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 8 텍스트 분석

## 07 문서 군집화 소개와 실습(Opinion Review 데이터 세트)

### 문서 군집화 개념



*   Document Clustering
*   비슷한 텍스트 구성의 문서응 군집화(Clustering)하는 것
*   동일한 군집에 속하는 문서를 같은 카테고리 소속으로 분류할 수 있으므로 텍스트 분류 기반의 문서 분류와 유사
*   텍스트 분류 기반의 문서 분류는 사전에 결정 카테고리 값을 가진 학습 데이터가 필요한 데 반해, 문서 군집화는 학습 데이터 세트가 필요 없는 비지도 학습 기반으로 동작




### Opinion Review 데이터 세트를 이용한 문서 군집화 수행하기

In [10]:
import pandas as pd
import glob ,os

# 아래는 제 컴퓨터에서 압축 파일을 풀어 놓은 디렉토리이니, 여러분의 디렉토리를 설정해 주십시요
path = r'/content/drive/MyDrive/ESAA/24-1/OpinosisDataset1.0/topics'
# path로 지정한 디렉토리 밑에 있는 모든 .data 파일들의 파일명을 리스트로 취합
all_files = glob.glob(os.path.join(path, "*.data"))
filename_list = []
opinion_text = []

# 개별 파일들의 파일명은 filename_list 리스트로 취합,
# 개별 파일들의 파일내용은 DataFrame로딩 후 다시 string으로 변환하여 opinion_text 리스트로 취합
for file_ in all_files:
    # 개별 파일을 읽어서 DataFrame으로 생성
    df = pd.read_table(file_,index_col=None, header=0,encoding='latin1')

    # 절대경로로 주어진 file 명을 가공. 만일 Linux에서 수행시에는 아래 \\를 / 변경. 맨 마지막 .data 확장자도 제거
    filename_ = file_.split('/')[-1]
    filename = filename_.split('.')[0]

    #파일명 리스트와 파일내용 리스트에 파일명과 파일 내용을 추가.
    filename_list.append(filename)
    opinion_text.append(df.to_string())

# 파일명 리스트와 파일내용 리스트를  DataFrame으로 생성
document_df = pd.DataFrame({'filename':filename_list, 'opinion_text':opinion_text})
document_df.head()

,filename,opinion_text
0,battery-life_ipod_nano_8gb,...
1,bathroom_bestwestern_hotel_sfo,...
2,accuracy_garmin_nuvi_255W_gps,...
3,battery-life_amazon_kindle,...
4,display_garmin_nuvi_255W_gps,...


In [11]:
from nltk.stem import WordNetLemmatizer
import nltk
import string
nltk.download('punkt')
nltk.download('wordnet')

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
lemmar = WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english' , \
                             ngram_range=(1,2), min_df=0.05, max_df=0.85 )

#opinion_text 컬럼값으로 feature vectorization 수행
feature_vect = tfidf_vect.fit_transform(document_df['opinion_text'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(


In [13]:
from sklearn.cluster import KMeans

# 5개 집합으로 군집화 수행. 예제를 위해 동일한 클러스터링 결과 도출용 random_state=0
km_cluster = KMeans(n_clusters=5, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [14]:
document_df['cluster_label'] = cluster_label
document_df.head()

,filename,opinion_text,cluster_label
0,battery-life_ipod_nano_8gb,...,1
1,bathroom_bestwestern_hotel_sfo,...,3
2,accuracy_garmin_nuvi_255W_gps,...,0
3,battery-life_amazon_kindle,...,1
4,display_garmin_nuvi_255W_gps,...,0


In [15]:
# cluster_label=0인 데이터 세트 : 차량용 네비게이션에 대한 리뷰로 군집화
document_df[document_df['cluster_label']==0].sort_values(by='filename')

,filename,opinion_text,cluster_label
2,accuracy_garmin_nuvi_255W_gps,...,0
9,buttons_amazon_kindle,...,0
10,directions_garmin_nuvi_255W_gps,...,0
4,display_garmin_nuvi_255W_gps,...,0
8,eyesight-issues_amazon_kindle,...,0
13,features_windows7,...,0
14,fonts_amazon_kindle,...,0
28,navigation_amazon_kindle,...,0
29,satellite_garmin_nuvi_255W_gps,...,0
37,screen_garmin_nuvi_255W_gps,...,0


In [16]:
# cluster_label=1인 데이터 세트 : 포터블 전자기기에 대한 리뷰로 군집화
document_df[document_df['cluster_label']==1].sort_values(by='filename')

,filename,opinion_text,cluster_label
3,battery-life_amazon_kindle,...,1
0,battery-life_ipod_nano_8gb,...,1
6,battery-life_netbook_1005ha,...,1
22,keyboard_netbook_1005ha,...,1
26,performance_netbook_1005ha,...,1
38,screen_ipod_nano_8gb,...,1
34,screen_netbook_1005ha,...,1
35,size_asus_netbook_1005ha,...,1
43,sound_ipod_nano_8gb,headphone jack i got a clear case for it a...,1
44,speed_windows7,...,1


In [17]:
# cluster_label=2인 데이터 세트 : 자동차에 대한 리뷰로 군집화
document_df[document_df['cluster_label']==2].sort_values(by='filename')

,filename,opinion_text,cluster_label
20,gas_mileage_toyota_camry_2007,...,2
18,mileage_honda_accord_2008,...,2


In [18]:
# cluster_label=3인 데이터 세트 : 대부분 호텔에 대한 리뷰로 군집화
document_df[document_df['cluster_label']==3].sort_values(by='filename')

,filename,opinion_text,cluster_label
1,bathroom_bestwestern_hotel_sfo,...,3
15,food_holiday_inn_london,...,3
12,food_swissotel_chicago,...,3
11,free_bestwestern_hotel_sfo,...,3
17,location_bestwestern_hotel_sfo,...,3
21,location_holiday_inn_london,...,3
24,parking_bestwestern_hotel_sfo,...,3
25,price_holiday_inn_london,...,3
32,room_holiday_inn_london,...,3
31,rooms_bestwestern_hotel_sfo,...,3


In [19]:
# cluster_label=4인 데이터 세트 : 킨들 대한 리뷰로 군집화
document_df[document_df['cluster_label']==4].sort_values(by='filename')

,filename,opinion_text,cluster_label
5,comfort_honda_accord_2008,...,4
7,comfort_toyota_camry_2007,...,4
19,interior_honda_accord_2008,...,4
16,interior_toyota_camry_2007,...,4
23,performance_honda_accord_2008,...,4
27,price_amazon_kindle,...,4
33,quality_toyota_camry_2007,...,4
41,seats_honda_accord_2008,...,4
47,transmission_toyota_camry_2007,...,4


In [20]:
# 3개의 집합으로 군집화
km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

# 소속 군집을 cluster_label 칼럼으로 할당하고 cluster_label 값으로 정렬
document_df['cluster_label'] = cluster_label
document_df.sort_values(by='cluster_label')

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,filename,opinion_text,cluster_label
0,battery-life_ipod_nano_8gb,...,0
48,voice_garmin_nuvi_255W_gps,...,0
46,speed_garmin_nuvi_255W_gps,...,0
44,speed_windows7,...,0
43,sound_ipod_nano_8gb,headphone jack i got a clear case for it a...,0
38,screen_ipod_nano_8gb,...,0
37,screen_garmin_nuvi_255W_gps,...,0
35,size_asus_netbook_1005ha,...,0
34,screen_netbook_1005ha,...,0
29,satellite_garmin_nuvi_255W_gps,...,0


### 군집별 핵심 단어 추출하기

In [21]:
cluster_centers = km_cluster.cluster_centers_
print('cluster_centers shape :', cluster_centers.shape)
print(cluster_centers)

cluster_centers shape : (3, 4611)
[[0.01005322 0.         0.         ... 0.00706287 0.         0.        ]
 [0.         0.00099499 0.00174637 ... 0.         0.00183397 0.00144581]
 [0.         0.00092551 0.         ... 0.         0.         0.        ]]


In [22]:
# 군집별 top n 핵심 단어, 그 단어의 중심 위치 상댓값, 대상 파일명을 반환함.
def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num,
                        top_n_features=10):
  cluster_details={}

  # cluster_centers array 값이 큰 순으로 정렬된 인덱스 값을 반환
  # 군집 중심점 별 할당된 word 피처들의 거리값이 큰 순으로 값을 구하기 위함
  centroid_feature_ordered_ind=cluster_model.cluster_centers_.argsort()[:, ::-1]

  # 개별 군집별로 반복하며 핵심 단어, 그 단어의 중심 위치 상댓값, 대상 파일명 입력
  for cluster_num in range(clusters_num):
    # 개별 군집별 정보를 담을 데이터 초기화
    cluster_details[cluster_num]={}
    cluster_details[cluster_num]['cluster']=cluster_num

    # cluster_centers_.argsort()로 구한 인덱스를 이용해 top n 피처 단어를 구함
    top_feature_indexes=centroid_feature_ordered_ind[cluster_num, :top_n_features]
    top_features=[feature_names[ind] for ind in top_feature_indexes]

    # top_feature_indexes를 이용해 해당 피처 단어의 중심 위치 상댓값 구함.
    top_feature_values=cluster_model.cluster_centers_[cluster_num,
                                                      top_feature_indexes].tolist()

    # cluster_details 딕셔너리 객체에 개별 군집별 핵심단어와 중심위치 상댓값, 해당 파일명 입력
    cluster_details[cluster_num]['top_features']=top_features
    cluster_details[cluster_num]['top_features_value']=top_feature_values
    filenames=cluster_data[cluster_data['cluster_label']==cluster_num]['filename']
    filenames=filenames.values.tolist()

    cluster_details[cluster_num]['filenames']=filenames

  return cluster_details

In [23]:
def print_cluster_details(cluster_details):
  for cluster_num, cluster_detail in cluster_details.items():
    print('####### Cluster {0}'.format(cluster_num))
    print('Top features:', cluster_detail['top_features'])
    print('Reviews 파일명 :', cluster_detail['filenames'][:7])
    print('===============================================')

In [24]:
feature_names = tfidf_vect.get_feature_names_out()
cluster_details = get_cluster_details(cluster_model = km_cluster, cluster_data = document_df,
                                      feature_names = feature_names, clusters_num = 3, top_n_features = 10)
print_cluster_details(cluster_details)

####### Cluster 0
Top features: ['screen', 'battery', 'keyboard', 'battery life', 'life', 'kindle', 'direction', 'video', 'size', 'voice']
Reviews 파일명 : ['battery-life_ipod_nano_8gb', 'accuracy_garmin_nuvi_255W_gps', 'battery-life_amazon_kindle', 'display_garmin_nuvi_255W_gps', 'battery-life_netbook_1005ha', 'eyesight-issues_amazon_kindle', 'buttons_amazon_kindle']
####### Cluster 1
Top features: ['room', 'hotel', 'service', 'staff', 'food', 'location', 'bathroom', 'clean', 'price', 'parking']
Reviews 파일명 : ['bathroom_bestwestern_hotel_sfo', 'free_bestwestern_hotel_sfo', 'food_swissotel_chicago', 'food_holiday_inn_london', 'location_bestwestern_hotel_sfo', 'location_holiday_inn_london', 'parking_bestwestern_hotel_sfo']
####### Cluster 2
Top features: ['interior', 'seat', 'mileage', 'comfortable', 'gas', 'gas mileage', 'transmission', 'car', 'performance', 'quality']
Reviews 파일명 : ['comfort_honda_accord_2008', 'comfort_toyota_camry_2007', 'interior_toyota_camry_2007', 'mileage_honda_acc